In [1]:
import json
from collections import defaultdict
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
import numpy as np
import random
import gzip
import dateutil.parser
import math
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
answers = {}

In [3]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [4]:
### Question 1

In [5]:
f = gzip.open("./../data/fantasy_10000.json.gz")
dataset = []
for l in f:
    dataset.append(json.loads(l))

In [6]:
# Get max review length
MAX_REVIEW_LENGTH = max(list(map(lambda x: len(x['review_text']), dataset)))
MAX_REVIEW_LENGTH

14306

In [7]:
def feature(datum):
    return len(datum['review_text'])/MAX_REVIEW_LENGTH

In [8]:
# Filter down and add intercept
X = list(map(feature, dataset))
X = list(map(lambda x: [1, x], X))
Y = list(map(lambda x: x['rating'], dataset))

In [9]:
# Train model and obtain coef and MSE
reg = linear_model.LinearRegression().fit(X, Y)

theta = reg.coef_
MSE = mean_squared_error(Y, reg.predict(X))

In [10]:
answers['Q1'] = [theta[0], theta[1], MSE]

In [11]:
assertFloatList(answers['Q1'], 3)

In [12]:
### Question 2

In [13]:
for d in dataset:
    t = dateutil.parser.parse(d['date_added'])
    d['parsed_date'] = t

In [14]:
# Clean dataset
df = pd.DataFrame(dataset)
df['parsed_date'] = pd.to_datetime(df['parsed_date'], format='%Y-%m-%d %H:%M:%S%z', utc=True)
df['weekday'] = df['parsed_date'].dt.strftime('%A')
df['month'] = df['parsed_date'].dt.strftime('%B')
df['review_text_prop'] = list(map(lambda x: len(x['review_text'])/MAX_REVIEW_LENGTH, dataset))
one_hot_df = pd.get_dummies(df, columns=['weekday', 'month'], prefix=['weekday', 'month'])

one_hot_df = one_hot_df.drop(columns=['user_id', 'book_id', 'review_id', 'date_added', 'date_updated', 'read_at', 'started_at', 'n_votes', 'n_comments', 'review_text', 'parsed_date', 'weekday_Monday', 'month_January'])
df = df.drop(columns=['user_id', 'book_id', 'review_id', 'date_added', 'date_updated', 'read_at', 'started_at', 'n_votes', 'n_comments', 'review_text', 'parsed_date'])

In [15]:
# Obtain Features
X = one_hot_df[one_hot_df.columns.difference(['rating'])]
Y = one_hot_df['rating']

In [16]:
answers['Q2'] = [np.append(X.loc[0].values, [1]),np.append(X.loc[1].values, [1])]

In [17]:
assertFloatList(answers['Q2'][0], 19)
assertFloatList(answers['Q2'][1], 19)

In [18]:
### Question 3

In [19]:
# Clean datasets
df['weekday'] = LabelEncoder().fit_transform(df['weekday'])
df['month'] = LabelEncoder().fit_transform(df['month'])
df['intercept'] = 1
one_hot_df['intercept'] = 1

In [20]:
# Obtain features
direct_X = df[df.columns.difference(['rating'])]
direct_Y = df['rating']
one_hot_X = one_hot_df[one_hot_df.columns.difference(['rating'])]
one_hot_Y = one_hot_df['rating']

In [21]:
# Train models
direct_reg = linear_model.LinearRegression().fit(direct_X, direct_Y)
one_hot_reg = linear_model.LinearRegression().fit(one_hot_X, one_hot_Y)

In [22]:
# Obtain MSEs
mse2 = mean_squared_error(direct_Y, direct_reg.predict(direct_X))
mse3 = mean_squared_error(one_hot_Y, one_hot_reg.predict(one_hot_X))

In [23]:
answers['Q3'] = [mse2, mse3]

In [24]:
assertFloatList(answers['Q3'], 2)

In [25]:
### Question 4

In [27]:
random.seed(0)
random.shuffle(dataset)

In [28]:
X2 = [feature(d) for d in dataset]
X3 = [feature3(d) for d in dataset]
Y = [d['rating'] for d in dataset]

NameError: name 'feature3' is not defined

In [ ]:
train2, test2 = X2[:len(X2)//2], X2[len(X2)//2:]
train3, test3 = X3[:len(X3)//2], X3[len(X3)//2:]
trainY, testY = Y[:len(Y)//2], Y[len(Y)//2:]

In [ ]:
answers['Q4'] = [test_mse2, test_mse3]

In [ ]:
assertFloatList(answers['Q4'], 2)

In [ ]:
### Question 5

In [ ]:
f = open("beer_50000.json")
dataset = []
for l in f:
    dataset.append(eval(l))

In [ ]:
X = 
y = 

In [ ]:
TP = 
TN = 
FP = 
FN = 

In [ ]:
BER = 

In [ ]:
answers['Q5'] = [TP, TN, FP, FN, BER]

In [ ]:
assertFloatList(answers['Q5'], 5)

In [ ]:
### Question 6

In [ ]:
precs = []

In [ ]:
for k in [1,100,1000,10000]:
    # ...

In [ ]:
answers['Q6'] = precs

In [ ]:
assertFloatList(answers['Q6'], 4)

In [ ]:
### Question 7

In [ ]:
its_test_MSE = 1000

In [ ]:
answers['Q7'] = ["Description of my solution", its_test_MSE]

In [ ]:
f = open("answers_hw1.txt", 'w')
f.write(str(answers) + '\n')
f.close()